# Advent of Code 2021 - Day 18

In [291]:
import java.io.File

val input: List<String> = File("Day18.input.txt")
    .bufferedReader()
    .readLines()

## Part 1

Difficult to simplify, read the problem on the site.

In [292]:
// import kotlin.collections.ArrayDeque

sealed interface Node
object Comma : Node
object LeftBracket : Node
object RightBracket : Node
@JvmInline value class Literal(val value: Int) : Node

// sealed interface SnailfishNumber : Node
// @JvmInline value class SnailfishLiteral(val value: Int) : SnailfishNumber {
//     override fun toString() = "$value"
// }
// data class SnailfishPair(val left: SnailfishNumber, val right: SnailfishNumber) : SnailfishNumber {
//     override fun toString() = "[$left,$right]"
// }

fun String.nodeIterator() = object : Iterator<Node> {
    private var iterator = this@nodeIterator.toList().listIterator()

    override fun hasNext(): Boolean = iterator.hasNext()

    override fun next(): Node = iterator.next().let { char ->
        when {
            char == ',' -> Comma
            char == '[' -> LeftBracket
            char == ']' -> RightBracket
            // char.isDigit() -> SnailfishLiteral((char + takeWhileDigit()).toInt())
            char.isDigit() -> Literal((char + takeWhileDigit()).toInt())
            else -> TODO()
        }
    }

    private fun takeWhileDigit(): String {
        if (!iterator.hasNext()) return ""

        var digits = ""
        var next = iterator.next()
        while (next.isDigit()) {
            digits += next
            next = iterator.next()
        }
        iterator.previous() 
        
        return digits
    }
}

// fun String.toSnailfishPair(): SnailfishPair =
//     ArrayDeque<Node>()
//         .apply {
//             var iterator = nodeIterator()
//             while (iterator.hasNext()) {
//                 val node = iterator.next()
//                 when (node) {
//                     Comma, LeftBracket, is SnailfishLiteral -> addFirst(node)
//                     RightBracket -> {
//                         val right = removeFirst() as SnailfishNumber
//                         val comma = removeFirst() as Comma
//                         val left = removeFirst() as SnailfishNumber
//                         val bracket = removeFirst() as LeftBracket
//                         addFirst(SnailfishPair(left, right))
//                     }
//                 }
//             }
//         }
//         .first() as SnailfishPair

// operator fun SnailfishPair.plus(other: SnailfishPair): SnailfishPair = SnailfishPair(this,other)

// input.map { it.toSnailfishPair() }

### Notes

Tried for a really long time to parse the number using recursion before giving up and doing it iteratively. The lack of proper union types was also pretty frustrating since `SnailfishNumber` needed to implement `Node` to safely parse despite them ideally existing independently.

So after finally getting through parsing and creating the final `SnailfishPair`, implementing explode ended up being really complicated as it was difficult to first identify what you needed to modify and then to subsequently "explode" it. I recognized really (really, really) late that this structure was actually just a binary tree.

Since this is a binary tree, a `LinkedList` might make this problem more manageable, or maybe just modifying it is the raw string...

## Part 1 - Round 2

Some parts from my first attempt were still useful (fortunately) so to keep a record of my efforts, the parts I don't use are commented out.

In [293]:
import java.util.ArrayDeque
import java.util.LinkedList

fun Iterator<Node>.toLinkedList() = LinkedList<Node>().apply {
    this@toLinkedList.forEachRemaining { add(it) }
}

fun LinkedList<Node>.asString() = joinToString("") {
    when (it) {
        Comma -> ","
        LeftBracket -> "["
        RightBracket -> "]"
        is Literal -> it.value.toString()
        else -> TODO()
    }
}

data class LiteralPair(
    val leftBracket: LeftBracket = LeftBracket, 
    val left: Literal, 
    val comma: Comma = Comma, 
    val right: Literal, 
    val rightBracket: RightBracket = RightBracket,
    val at: IntRange = 0..0,
) : List<Node> by listOf(leftBracket, left, comma, right, rightBracket)

fun LinkedList<Node>.literalPairAt(i: Int) = 
    LiteralPair(get(i) as LeftBracket, get(i + 1) as Literal, get(i + 2) as Comma, get(i +3) as Literal, get(i + 4) as RightBracket, i..i+4)

operator fun Literal.plus(other: Literal) = Literal(value + other.value)
operator fun Literal.times(other: Literal) = Literal(value * other.value)

fun LinkedList<Node>.explode(pair: LiteralPair): LinkedList<Node> = apply {
    for (index in pair.at.start downTo 0) {
        when (val node = get(index)) { is Literal -> { set(index, node + pair.left); break } }
    }
    for (index in pair.at.endInclusive..size - 1) {
        when (val node = get(index)) { is Literal -> { set(index, node + pair.right); break } }
    }
    for (index in pair.at) { removeAt(pair.at.start) }
    add(pair.at.start, Literal(0))
}

fun LinkedList<Node>.split(at: Int): LinkedList<Node> = apply {
    (get(at) as Literal).let {
        removeAt(at)
        addAll(at, LiteralPair(left = Literal(it.value / 2), right = Literal(it.value - (it.value / 2))))
    }
}

fun LinkedList<Node>.explode(): Boolean {
    var depth = 0
    forEachIndexed { index, node ->
        when (node) {
            LeftBracket -> depth++
            RightBracket -> depth--
            else -> Unit
        }

        if (depth == 5) {
            explode(literalPairAt(index))
            return true
        }
    }
    return false
}

fun LinkedList<Node>.split(): Boolean {
    forEachIndexed { index, node -> 
        if (node is Literal && node.value >= 10) {
            split(index)
            return true
        }
    }
    return false
}

operator fun LinkedList<Node>.plus(other: LinkedList<Node>) = apply {
    addFirst(LeftBracket)
    add(Comma)
    addAll(other)
    add(RightBracket)
    while (explode() || split()) Unit
}

fun LinkedList<Node>.magnitude(): Int {
    val stack = ArrayDeque<MutableList<Literal>>()
    forEach { node ->
        when (node) {
            LeftBracket -> stack.addFirst(mutableListOf())
            is Literal -> stack.first().add(node)
            RightBracket -> stack.removeFirst()
                .let { (left, right) -> (Literal(3) * left + Literal(2) * right) }
                .let { if (stack.isEmpty()) return it.value else stack.first().add(it) }
            else -> Unit
        }
    }
    return 0
}

input
    .map { it.nodeIterator() }
    .map { it.toLinkedList() }
    .reduce { acc, nodes -> acc + nodes }
    .magnitude()

4088

### Notes

So the problem got significantly easier after treating the input as a `LinkedList`. To `explode`, it was straightforward to figure out the left and right numbers to explode "into" since the `LinkedList` provides forwards and backwards traversal. Deletion and insertion similarly was easy as long as we kept the index. To `split` was achieved similarly. Calculating the magnitude required some thought, but we can utilize a stack to keep track of which numbers belong to which pair.

Actually pretty happy with the `Iterator` implementation in the initial version since ending up with parsed / typed nodes was really helpful.

## Part 2

What's the largest magnitude of any two snailfish numbers?

In [294]:
input.maxOf { it1 ->
    input.maxOf { it2 ->
        if (it1 == it2) 0
        else (it1.nodeIterator().toLinkedList() + it2.nodeIterator().toLinkedList()).magnitude()
    }
}

4536

### Notes

We just need to add every number with every other number and find the max, if part 1 works correctly, then this is looping twice.